# Virtual screening

Ligand-based virtual screening (LBVS) involves predicting bioactivity of large collections of compounds. It involves multiple steps, which are quite easy to perform with scikit-fingerprints.

---

## Molecular filters

TODO quick word about molecular filters, what they are for and why we want to use them.

### Physicochemical filters

Todo quick word

In [1]:
from skfp.filters import LipinskiFilter

"""TODO here:
    - Example implementation without skfp
    - Demonstrate use of filters
    - Show what to do if we want to filter dataset with labels. Or how to work on a dataframe. Example problem.
        - If we don't know but we can check skfp docs!
        - Look! There's a nice `return_indicators` argument
    - Show allowed violations and similar parameters of popular filters
"""

'TODO here:\n    demonstrate use of filters\n    show what to do if we want to filter dataset with labels\n'

### Substructural filters

TODO quick word

In [ ]:
"""TODO here:
    - Show example filter
    - Show how to get feature names
"""

---

## Data splits

Quick word if someone isn't familiar

In [ ]:
"""TODO here:
    - Show scaffold split without skfp
    - Remind that we can use load_X_splits
    - Show manual split computation with skfp
"""

---

## Similarity searching & evaluation metrics



### Similarity search

quick word

In [ ]:
"""TODO here:
    - Similarity search without skfp
    - Similarity search - find similar molecules
    - KNN and compatibility
    - Bulk similarity computation
    - Times
"""

### Evaluation metrics

In [3]:
"""TODO here:
    - Show example metric. like auroc
    - Show multioutput
    - Make scorer for grid search using our metric. Like mcc
"""

'TODO here:\n    - Show example metric. like auroc\n    - Show multioutput\n    - Make scorer for grid search using our metric. Like mcc\n'

---